In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pylab as plt
import progressbar

np.random.seed(42)

In [ ]:
# translate SNANA types
types_names = {90:'Ia', 67: '91bg', 52:'Iax', 42:'II', 62:'Ibc', 
               95: 'SLSN', 15:'TDE', 64:'KN', 88:'AGN', 92:'RRL', 65:'M-dwarf',
               16:'EB',53:'Mira', 6:'MicroL', 991:'MicroLB', 992:'ILOT', 
               993:'CART', 994:'PISN',995:'MLString'}

SNANA_types = {90:11, 62:{1:3, 2:13}, 42:{1:2, 2:12, 3:14},
               67:41, 52:43, 64:51, 95:60, 994:61, 992:62,
               993:63, 15:64, 88:70, 92:80, 65:81, 16:83,
               53:84, 991:90, 6:{1:91, 2:93}}

SNANA_names = {11: 'Ia', 3:'Ibc', 13: 'Ibc', 2:'II', 12:'II', 14:'II',
               41: '91bg', 43:'Iax', 51:'KN', 60:'SLSN', 61:'PISN', 62:'ILOT',
               63:'CART', 64:'TDE', 70:'AGN', 80:'RRL', 81:'M-dwarf', 83:'EB',
               84:'Mira', 90:'MicroLB', 91:'MicroL', 93:'MicroL'}

In [ ]:
# this needs to be run just once
output_root = '/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data3/'

for field in ['DDF', 'WFD']:
    for version in range(10):
        for nobjs in [1500, 3000, 6000]:

            # create directory structure
            dir_list = [output_root + field + '/',
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/',
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/cospar/',
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/fitres/', 
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/M0DIF/',
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/posteriors/',
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/posteriors/csv/',
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/posteriors/pkl',
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/posteriors/trace',
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/samples/',
                        output_root + field + '/results/v' + str(version) + '/' + str(nobjs) + '/stan_input/',
                        output_root + field + '/results/v' + str(version) +'/' + str(nobjs) +  '/stan_summary/',
                      ]

            for name in dir_list:
                if not os.path.isdir(name):
                    os.makedirs(name)


In [ ]:
# read zenodo metadata
fname = '/media/RESSPECT/data/PLAsTiCC/PLAsTiCC_zenodo/plasticc_test_metadata.csv'
test_metadata = pd.read_csv(fname)

# separate fields
ddf_flag = test_metadata['ddf_bool'].values == 1
ids_ddf = test_metadata['object_id'].values[ddf_flag]
ids_wfd = test_metadata['object_id'].values[~ddf_flag]

# Create perfect samples

In [ ]:
v = '0'
nobjs = 3000
data_dir = output_root + field + '/results/v' + v + '/' + str(nobjs) + '/samples/'

## For DDF

In [ ]:
# read all Ias in DDF
salt2_Ia_DDF = pd.read_csv(output_root + 'DDF/SALT2_fit/Ia/fitres/master_fitres_1.fitres', 
                           comment='#', delim_whitespace=True)
salt2_Ia_DDF['zHD'] = salt2_Ia_DDF['SIM_ZCMB']          # requirement of so SALT2mu can work

# choose number of versions of the same sample to generate
v = -1

for i in range(1):
    perfect_Ia_DDF = salt2_Ia_DDF.sample(n=nobjs, replace=False)
    #perfect_Ia_DDF.to_csv(data_dir + 'DDF/v' + str(i) +  '/samples/perfect' + \
    #                      str(nobjs) + '.csv', sep=' ', index=False)

In [ ]:
np.unique(salt2_Ia_DDF['SIM_TYPE_INDEX'].values)

In [ ]:
salt2_Ia_DDF.shape

In [ ]:
plt.figure(figsize=(18,5))
plt.subplot(1,3,1)
plt.hist(perfect_Ia_DDF['x1'] - perfect_Ia_DDF['SIM_x1'], bins=30)
plt.xlabel('x1 - SIM_x1')

plt.subplot(1,3,2)
plt.hist(perfect_Ia_DDF['c'] - perfect_Ia_DDF['SIM_c'], bins=20 )
plt.xlabel('c - SIM_c')

mask = perfect_Ia_DDF['mB'] - perfect_Ia_DDF['SIM_mB'] < 2
plt.subplot(1,3,3)
plt.hist(perfect_Ia_DDF['mB'][mask] - perfect_Ia_DDF['SIM_mB'][mask], bins=20)
plt.xlabel('mB - SIM_mB')
plt.show()

## For WFD

In [ ]:
# read all Ias in WFD
fnames_Ia = glob.glob(output_root + 'WFD/SALT2_fit/Ia/fitres/master_fitres_*.fitres')

salt2_WFD = []

for name in fnames_Ia:
    try:
        fitres_temp = pd.read_csv(name, delim_whitespace=True, 
                                  comment='#')
        fitres_temp['zHD'] = fitres_temp['SIM_ZCMB']
        salt2_WFD.append(fitres_temp)
    except ValueError:
        pass
    else:
        pass

salt2_Ia_WFD = pd.concat(salt2_WFD, ignore_index=True)

# choose number of versions of the same sample to generate
v = 1

for i in range(v):
    perfect_Ia_WFD = salt2_Ia_WFD.sample(n=nobjs, replace=False)
    mask = perfect_Ia_WFD['mB'] - perfect_Ia_WFD['SIM_mB'] < 10000
    #perfect_Ia_WFD.to_csv(output_root + 'WFD/v' + str(i) + '/samples/perfect' + \
    #                      str(nobjs) + '.csv', sep=' ', index=False)

In [ ]:
np.unique(salt2_Ia_WFD['SIM_TYPE_INDEX'].values)

In [ ]:
salt2_Ia_WFD.shape

In [ ]:
plt.figure(figsize=(18,5))
plt.subplot(1,3,1)
plt.hist(perfect_Ia_WFD['x1'] - perfect_Ia_WFD['SIM_x1'], bins=30)
plt.xlabel('x1 - SIM_x1')

plt.subplot(1,3,2)
plt.hist(perfect_Ia_WFD['c'] - perfect_Ia_WFD['SIM_c'], bins=20 )
plt.xlabel('c - SIM_c')

mask = perfect_Ia_WFD['mB'] - perfect_Ia_WFD['SIM_mB'] < 10000
plt.subplot(1,3,3)
plt.hist(perfect_Ia_WFD['mB'][mask] - perfect_Ia_WFD['SIM_mB'][mask], bins=30)
plt.xlabel('mB - SIM_mB')
#plt.savefig('dist_WFD_perfct3000.png')
plt.show()

# Create Random samples

## For DDF

In [ ]:
# list of classes surviving SALT2 fit
surv_class_DDF = os.listdir(output_root + 'DDF/SALT2_fit/')

# read all SALT2 fit results for DDF
all_DDF = []

for obj_type in surv_class_DDF: 
    if obj_type == 'Ia':
        data_temp = pd.read_csv(output_root + 'DDF/SALT2_fit/' + \
                            obj_type + '/fitres/master_fitres_1.fitres', 
                            comment='#', delim_whitespace=True)
        data_temp['zHD'] = data_temp['SIM_ZCMB']
        data_temp.fillna(-99, inplace=True)
        all_DDF.append(data_temp)
        print(obj_type, np.unique(data_temp['SIM_TYPE_INDEX'].values), data_temp.shape[0])
        
    elif obj_type not in ['Ibc', 'II']:
        try:
            data_temp = pd.read_csv(output_root + 'DDF/SALT2_fit/' + \
                                    obj_type + '/fitres/master_fitres_1.fitres', 
                                    comment='#', delim_whitespace=True)
            data_temp['zHD'] = data_temp['SIM_ZCMB']
            data_temp.fillna(-99, inplace=True)
            all_DDF.append(data_temp)
            print(obj_type, np.unique(data_temp['SIM_TYPE_INDEX'].values), data_temp.shape[0])
        except:
            pass
        
    elif obj_type == 'Ibc':
        for subtype in ['13', '3']:
            data_temp = pd.read_csv(output_root + 'DDF/SALT2_fit/' + \
                                    obj_type + '/fitres/master_fitres_' + subtype \
                                    + '_1.fitres', 
                                    comment='#', delim_whitespace=True)
        
            data_temp['zHD'] = data_temp['SIM_ZCMB']
            data_temp.fillna(-99, inplace=True)
            all_DDF.append(data_temp)
            print(obj_type, np.unique(data_temp['SIM_TYPE_INDEX'].values), data_temp.shape[0])
    elif obj_type == 'II':
        for subtype in ['12', '14', '2']:
            data_temp = pd.read_csv(output_root + 'DDF/SALT2_fit/' + \
                                    obj_type + '/fitres/master_fitres_' + subtype \
                                    + '_1.fitres', 
                                    comment='#', delim_whitespace=True)
        
            data_temp['zHD'] = data_temp['SIM_ZCMB']
            data_temp.fillna(-99, inplace=True)
            all_DDF.append(data_temp)
            print(obj_type, np.unique(data_temp['SIM_TYPE_INDEX'].values), data_temp.shape[0])
    
all_surv_DDF = pd.concat(all_DDF, ignore_index=True)
all_surv_DDF.fillna(-99, inplace=True)

# choose number of versions of the same sample to generate
v = 1

for i in range(v):
    random_DDF = all_surv_DDF.sample(n=nobjs, replace=False)
    #random_DDF.to_csv(output_root + 'DDF/results/v' + str(i) + '/' + str(nobjs) + '/samples/random' + \
    #                  str(nobjs) + '.csv', sep=' ', index=False)

In [ ]:
snana_numbers_ddf, freq_ddf = np.unique(all_surv_DDF['SIM_TYPE_INDEX'].values, 
                                        return_counts=True)

for i in range(len(freq_ddf)):
    print(SNANA_names[snana_numbers_ddf[i]], '\t', freq_ddf[i], 
          '\t', 100*freq_ddf[i]/all_surv_DDF.shape[0])

In [ ]:
# type II 
100*(freq_ddf[0] + freq_ddf[3] + freq_ddf[5])/all_surv_DDF.shape[0]

In [ ]:
# type Ibc
100*(freq_ddf[1]+freq_ddf[4])/all_surv_DDF.shape[0]

In [ ]:
all_surv_DDF.shape

## For WFD

In [ ]:
# list of classes surviving SALT2 fit
flist = glob.glob(output_root + 'WFD/SALT2_fit/*/fitres/master_fitres_*.fitres')

# read all SALT2 fit results for WFD
all_WFD = []
for name in flist:
    try:
        data_temp = pd.read_csv(name, comment='#', delim_whitespace=True)
        data_temp['zHD'] = data_temp['SIM_ZCMB']
        data_temp.fillna(-99, inplace=True)
        all_WFD.append(data_temp)
        print(name, np.unique(data_temp['SIM_TYPE_INDEX'].values))
    except:
        pass
    
all_surv_WFD = pd.concat(all_WFD, ignore_index=True)
all_surv_WFD.fillna(-99, inplace=True)

# choose number of versions of the same sample to generate
v = 1

for i in range(v):
    random_WFD = all_surv_WFD.sample(n=nobjs, replace=False)
    #random_WFD.to_csv(output_root + 'WFD/results/v' + str(i) + '/' + str(nobjs) + '/samples/random' + \
    #                  str(nobjs) + '.csv', sep=' ', index=False)

In [ ]:
salt2_Ia_WFD.shape[0]/all_surv_WFD.shape[0]

In [ ]:
snana_numbers_wfd, freq_wfd = np.unique(all_surv_WFD['SIM_TYPE_INDEX'].values, return_counts=True)

for i in range(len(freq_wfd)):
    print(SNANA_names[snana_numbers_wfd[i]], '\t', freq_wfd[i], '\t', 100*freq_wfd[i]/all_surv_WFD.shape[0])

In [ ]:
# type II
100*(freq_wfd[0]+ freq_wfd[3] + freq_wfd[5])/all_surv_WFD.shape[0]

In [ ]:
# type Ibc
100*(freq_wfd[1] + freq_wfd[4])/all_surv_WFD.shape[0]

In [ ]:
all_surv_WFD.shape

# Create Fiducial samples

## For DDF

In [ ]:
# read results from avocado
fname_DDF = '/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/DDF/avocado/avocado_DDF.csv'
avocado_DDF = pd.read_csv(fname_DDF, names=['object_id','6','15','16','42','52','53','62','64','65','67','88',
                                           '90','92','95'], skiprows=1)

# determine final classification
class_final_DDF = []
for i in progressbar.progressbar(range(avocado_DDF.shape[0])):
    indx = np.argsort(avocado_DDF.iloc[i].values[1:])[-1]
    code = int(avocado_DDF.keys()[indx + 1])
    class_final_DDF.append(types_names[code])
class_final_DDF = np.array(class_final_DDF)

# get photometrically classified Ia
flag_class_Ia_DDF = class_final_DDF == 'Ia'
avocado_DDF_Ia = avocado_DDF[flag_class_Ia_DDF]

# get SALT2 fit for objs photometrically classified as Ia
avocado_DDF_Ia_fitres_flag = np.array([item in avocado_DDF_Ia['object_id'].values 
                                       for item in all_surv_DDF['CID'].values])
all_avocado_DDF_Ia = all_surv_DDF[avocado_DDF_Ia_fitres_flag]

# choose number of versions of the same sample to generate
v = 10

for i in range(v):
    fiducial_DDF = all_avocado_DDF_Ia.sample(n=nobjs, replace=False)
    fiducial_DDF.to_csv(output_root + 'DDF/results/v' + str(i) + '/' + str(nobjs) + '/samples/fiducial' + \
                      str(nobjs) + '.csv', sep=' ', index=False)

In [ ]:
output_root + 'DDF/results/v' + str(i) + '/' + str(nobjs) + '/samples/fiducial' + \
                      str(nobjs) + '.csv'

## For WFD

In [ ]:
# read results from avocado
fname_WFD =  '/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/WFD/avocado/avocado_WFD.csv'
avocado_WFD = pd.read_csv(fname_WFD, names=['object_id','6','15','16','42','52','53','62','64','65','67','88',
                                           '90','92','95'], skiprows=1)

# determine final classification
class_final_WFD = []
for i in progressbar.progressbar(range(avocado_WFD.shape[0])):
    indx = np.argsort(avocado_WFD.iloc[i].values[1:])[-1]
    code = int(avocado_WFD.keys()[indx + 1])
    class_final_WFD.append(types_names[code])
    
# get photometrically classified Ia
class_final_WFD = np.array(class_final_WFD)
flag_class_Ia_WFD = class_final_WFD == 'Ia'
avocado_WFD_Ia = avocado_WFD[flag_class_Ia_WFD]

# get SALT2 fit for objs photometrically classified as Ia
avocado_WFD_Ia_fitres_flag = np.array([item in avocado_WFD_Ia['object_id'].values 
                                       for item in all_surv_WFD['CID'].values])
all_avocado_WFD_Ia = all_surv_WFD[avocado_WFD_Ia_fitres_flag]

# choose number of versions of the same sample to generate
v = 10

for i in range(v):
    fiducial_WFD = all_avocado_WFD_Ia.sample(n=nobjs, replace=False)
    fiducial_WFD.to_csv(output_root + 'WFD/results/v' + str(i) + '/' + str(nobjs) + '/samples/fiducial' + \
                      str(nobjs) + '.csv', sep=' ', index=False)

In [ ]:
obj, freq = np.unique(fiducial_WFD['SIM_TYPE_INDEX'].values, return_counts=True)
for i in range(len(obj)):
    print(SNANA_names[obj[i]], '  --  ', freq[i]/fiducial_WFD.shape[0])

# Create single contaminant samples

## For DDF

In [ ]:
# levels of contamination
cont_DDF = {'II': [0.28, 0.25, 0.1, 0.05, 0.02, 0.01],
            'Ibc': [0.05, 0.02, 0.01],
            'Iax': [0.1, 0.05, 0.02, 0.01],
            'CART': [0.006],
            'SLSN': [0.001]}

complete_names ={'II': 'SNII', 'Ibc': 'SNIbc', 'Iax': 'SNIax', 'CART':'CART',
                 '91bg':'SNIa-91bg', 'AGN':'AGN', 'SLSN':'SLSN'}

# choose number of versions of the same sample to generate
v = 10

for i in range(v):
    for obj_class in list(cont_DDF.keys()):
        # read all contaminants surviving SALT2 fit
        if obj_class not in ['II', 'Ibc']:
            sample_cont = pd.read_csv(output_root + 'DDF/SALT2_fit/' + obj_class + '/fitres/master_fitres_1.fitres', comment='#',
                                  delim_whitespace=True)
        elif obj_class == 'Ibc':
            sample_cont_list = []
            for subset in  ['13', '3']:
                data_temp = pd.read_csv(output_root + 'DDF/SALT2_fit/' + \
                                        obj_class + '/fitres/master_fitres_' + subset \
                                        + '_1.fitres', 
                                        comment='#', delim_whitespace=True)
                sample_cont_list.append(data_temp)
                
            sample_cont = pd.concat(sample_cont_list, ignore_index=True) 
            
        elif obj_class == 'II':
            sample_cont_list = []
            for subset in ['12', '14', '2']:
                data_temp = pd.read_csv(output_root + 'DDF/SALT2_fit/' + \
                                        obj_class + '/fitres/master_fitres_' + subset \
                                        + '_1.fitres', 
                                        comment='#', delim_whitespace=True)
                
                sample_cont_list.append(data_temp)
                
            sample_cont = pd.concat(sample_cont_list, ignore_index=True) 
        
        sample_cont['zHD'] = sample_cont['SIM_ZCMB']
    
        for perc in cont_DDF[obj_class]:
            Ia_temp = salt2_Ia_DDF.sample(n=int((1 - perc) * nobjs), replace=False)
            cont_temp = sample_cont.sample(n=int(perc * nobjs), replace = False)
            sample_final = pd.concat([Ia_temp, cont_temp], ignore_index=True)
            sample_final.fillna(-99, inplace=True)
        
            #if obj_class not in ['CART', 'SLSN']:
            #    sample_final.to_csv(output_root + 'DDF/results/v' + str(i)  + '/' + str(nobjs) + '/samples/' + str(int(100 - 100 * perc)) + \
            #                        'SNIa' + str(int(100 * perc)) + complete_names[obj_class] + '.csv', 
            #                         sep=' ', index=False)
            #else:
            #    sample_final.to_csv(output_root + 'DDF/results/v' + str(i) + '/'+  str(nobjs) + '/samples/' + str(round(100 - 100 * perc, 1)) + \
            #                        'SNIa' + str(round(100 * perc, 1)) + complete_names[obj_class] + '.csv', 
            #                         sep=' ', index=False)

## For WFD

In [ ]:
# levels of contamination
cont_WFD = {'II': [0.28, 0.25, 0.1, 0.05, 0.02, 0.01],
            'Ibc': [0.1, 0.05, 0.02, 0.01],
            'Iax': [0.25, 0.1, 0.05, 0.02, 0.01],
            'SLSN': [0.15, 0.10, 0.05, 0.02, 0.01],
            '91bg': [0.1, 0.05, 0.02, 0.01],
            'AGN': [0.04, 0.02, 0.01],
            'PISN': [0.003],
             'ILOT': [0.006],
            'CART': [0.1, 0.05, 0.02, 0.01]}

complete_names ={'II': 'SNII', 'Ibc': 'SNIbc', 'Iax': 'SNIax', 'CART':'CART',
                 '91bg':'SNIa-91bg', 'AGN':'AGN', 'PISN':'PISN', 'ILOT':'ILOT', 'SLSN':'SLSN'}

for i in range(v):
    for obj_class in list(cont_WFD.keys()):
        # read all contaminants surviving SALT2 fit
        flist = glob.glob(output_root + 'WFD/SALT2_fit/' + obj_class + '/fitres/master_fitres_*.fitres')
        
        sample_cont = []
        for name in flist:
            
            try:
                temp_cont = pd.read_csv(name, comment='#', delim_whitespace=True)
                temp_cont['zHD'] = temp_cont['SIM_ZCMB']
                sample_cont.append(temp_cont)
                print(obj_class, perc, '  temp_cont.size = ', temp_cont.shape[0], '  ', 
                  str(np.unique(temp_cont['SIM_TYPE_INDEX'])) )
            except:
                pass
            
        if len(sample_cont) > 0:
            sample_cont2 = pd.concat(sample_cont, ignore_index=True)
    
            for perc in cont_WFD[obj_class]:                
                Ia_temp2 = salt2_Ia_WFD.sample(n=int((1-perc)*nobjs), replace=False)
                cont_temp2 = sample_cont2.sample(n=int(perc*nobjs), replace = False)
                sample_final = pd.concat([Ia_temp2, cont_temp2], ignore_index=True)
                sample_final.fillna(-99, inplace=True)
                if obj_class in ['Iax'] :
                    sample_final.to_csv(output_root + 'WFD/results/v' + str(i) + '/' + str(nobjs) + '/samples/' + str(int(100 - 100 * perc)) + \
                                       'SNIa' + str(int(100 * perc)) + complete_names[obj_class] + '.csv', 
                                    sep=' ', index=False)